### import

In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.0


### Checkpoint function

In [13]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir) # 해당 dir에 학습해둔 모델 있는지 확인
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0
    
def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [14]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N,28,28] -> [N,28,28,1]
    test_data = np.expand_dims(test_data, axis=-1) # [N,28,28] -> [N,28,28,1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

### Performance function

In [15]:
def loss_func(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                   y_true=labels,
                                                                   from_logits=True)) # softmax를 거치지 않을때
    return loss

def accuracy_func(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_func(model, images, labels)
    return tape.gradient(loss, model.variables)

### Model function

In [16]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units=label_dim, # 출력값의 크기
                                 use_bias=True, # 편향 사용 유무
                                 kernel_initializer=weight_init) # 가중치 초기화 함수

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate):
    return tf.keras.layers.Dropout(rate)

### Create model(class version)

In [18]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() 
        # Sigmoid : Xavier 초기화
        # Relu : He 초기화
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        
        for i in range(2):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=.5))
            
        self.model.add(dense(label_dim, weight_init))
        
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Create model(function version)

In [19]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=.5))
        
    model.add(dense(label_dim, weight_init))
    
    return model

### Define data & hyper-parameter

In [20]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10
train_flag = True

# Graph input using Dataset API
# prefetch : data 로드 시간을 줄이기 위해 미리 batch size 만큼 적재시킴
# drop_remainder : 마지막 남은 data를 drop 여부
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
                                                   shuffle(buffer_size=100000).\
                                                   prefetch(buffer_size=batch_size).\
                                                   batch(batch_size, drop_remainder=True) 


test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
                                                  shuffle(buffer_size=100000).\
                                                  prefetch(buffer_size=len(test_x)).\
                                                  batch(len(test_x))

### Define model & optimizer & writer

In [21]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [22]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
        
    # train phase
    with summary_writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_func(network, train_input, train_label)
                train_accuracy = accuracy_func(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_func(network, test_input, test_label)
                    
                tf.summary.scalar(name='train_lass', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))   
# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_func(network, test_input, test_label)
        
    print("test Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.4948, train_loss: 2.10985041, train_accuracy: 0.5000, test_Accuracy: 0.3620
Epoch: [ 0] [    1/  468] time: 0.6812, train_loss: 2.11913323, train_accuracy: 0.5234, test_Accuracy: 0.4022
Epoch: [ 0] [    2/  468] time: 0.8579, train_loss: 2.03499603, train_accuracy: 0.5234, test_Accuracy: 0.4828
Epoch: [ 0] [    3/  468] time: 1.0385, train_loss: 1.88483274, train_accuracy: 0.5391, test_Accuracy: 0.5368
Epoch: [ 0] [    4/  468] time: 1.2512, train_loss: 1.78676832, train_accuracy: 0.6641, test_Accuracy: 0.6205
Epoch: [ 0] [    5/  468] time: 1.4220, train_loss: 1.63764000, train_accuracy: 0.6562, test_Accuracy: 0.6858
Epoch: [ 0] [    6/  468] time: 1.6016, train_loss: 1.63432550, train_accuracy: 0.7109, test_Accuracy: 0.7250
Epoch: [ 0] [    7/  468] time: 1.7773, train_loss: 1.47785807, train_accuracy: 0.7969, test_Accuracy: 0.7408
Epoch: [ 0] [    8/  468] time: 1.9569,

Epoch: [ 0] [   74/  468] time: 14.7796, train_loss: 0.39134270, train_accuracy: 0.8828, test_Accuracy: 0.9081
Epoch: [ 0] [   75/  468] time: 15.0021, train_loss: 0.51667625, train_accuracy: 0.8828, test_Accuracy: 0.9076
Epoch: [ 0] [   76/  468] time: 15.2168, train_loss: 0.43735245, train_accuracy: 0.9141, test_Accuracy: 0.9061
Epoch: [ 0] [   77/  468] time: 15.4306, train_loss: 0.41352683, train_accuracy: 0.8984, test_Accuracy: 0.9071
Epoch: [ 0] [   78/  468] time: 15.6463, train_loss: 0.43517849, train_accuracy: 0.8750, test_Accuracy: 0.9092
Epoch: [ 0] [   79/  468] time: 15.8698, train_loss: 0.32101160, train_accuracy: 0.9219, test_Accuracy: 0.9100
Epoch: [ 0] [   80/  468] time: 16.1206, train_loss: 0.37837893, train_accuracy: 0.9062, test_Accuracy: 0.9118
Epoch: [ 0] [   81/  468] time: 16.3558, train_loss: 0.29837203, train_accuracy: 0.9297, test_Accuracy: 0.9143
Epoch: [ 0] [   82/  468] time: 16.6027, train_loss: 0.28235692, train_accuracy: 0.9531, test_Accuracy: 0.9168
E

Epoch: [ 0] [  148/  468] time: 31.0661, train_loss: 0.35295826, train_accuracy: 0.9375, test_Accuracy: 0.9355
Epoch: [ 0] [  149/  468] time: 31.2877, train_loss: 0.34315133, train_accuracy: 0.9219, test_Accuracy: 0.9363
Epoch: [ 0] [  150/  468] time: 31.4926, train_loss: 0.23170649, train_accuracy: 0.9453, test_Accuracy: 0.9358
Epoch: [ 0] [  151/  468] time: 31.6937, train_loss: 0.22262436, train_accuracy: 0.9609, test_Accuracy: 0.9349
Epoch: [ 0] [  152/  468] time: 31.8996, train_loss: 0.25190338, train_accuracy: 0.9453, test_Accuracy: 0.9345
Epoch: [ 0] [  153/  468] time: 32.1075, train_loss: 0.34770602, train_accuracy: 0.9375, test_Accuracy: 0.9344
Epoch: [ 0] [  154/  468] time: 32.3271, train_loss: 0.33750445, train_accuracy: 0.9375, test_Accuracy: 0.9346
Epoch: [ 0] [  155/  468] time: 32.5584, train_loss: 0.32535458, train_accuracy: 0.8984, test_Accuracy: 0.9359
Epoch: [ 0] [  156/  468] time: 32.7975, train_loss: 0.34733135, train_accuracy: 0.9375, test_Accuracy: 0.9382
E

Epoch: [ 0] [  222/  468] time: 47.0618, train_loss: 0.18786353, train_accuracy: 0.9688, test_Accuracy: 0.9458
Epoch: [ 0] [  223/  468] time: 47.2745, train_loss: 0.28505942, train_accuracy: 0.9375, test_Accuracy: 0.9455
Epoch: [ 0] [  224/  468] time: 47.5000, train_loss: 0.20387807, train_accuracy: 0.9531, test_Accuracy: 0.9456
Epoch: [ 0] [  225/  468] time: 47.7167, train_loss: 0.27628848, train_accuracy: 0.9219, test_Accuracy: 0.9453
Epoch: [ 0] [  226/  468] time: 47.9343, train_loss: 0.33343458, train_accuracy: 0.9297, test_Accuracy: 0.9434
Epoch: [ 0] [  227/  468] time: 48.1598, train_loss: 0.28506845, train_accuracy: 0.9531, test_Accuracy: 0.9434
Epoch: [ 0] [  228/  468] time: 48.3657, train_loss: 0.14867257, train_accuracy: 0.9688, test_Accuracy: 0.9416
Epoch: [ 0] [  229/  468] time: 48.5736, train_loss: 0.20161898, train_accuracy: 0.9688, test_Accuracy: 0.9398
Epoch: [ 0] [  230/  468] time: 48.7756, train_loss: 0.35574648, train_accuracy: 0.9062, test_Accuracy: 0.9378
E

Epoch: [ 0] [  296/  468] time: 62.9959, train_loss: 0.15446363, train_accuracy: 0.9688, test_Accuracy: 0.9514
Epoch: [ 0] [  297/  468] time: 63.2214, train_loss: 0.19193456, train_accuracy: 0.9844, test_Accuracy: 0.9517
Epoch: [ 0] [  298/  468] time: 63.4332, train_loss: 0.27432591, train_accuracy: 0.9375, test_Accuracy: 0.9512
Epoch: [ 0] [  299/  468] time: 63.6577, train_loss: 0.29336971, train_accuracy: 0.9688, test_Accuracy: 0.9502
Epoch: [ 0] [  300/  468] time: 63.8812, train_loss: 0.22120558, train_accuracy: 0.9453, test_Accuracy: 0.9491
Epoch: [ 0] [  301/  468] time: 64.0930, train_loss: 0.24986732, train_accuracy: 0.9531, test_Accuracy: 0.9499
Epoch: [ 0] [  302/  468] time: 64.3038, train_loss: 0.20163706, train_accuracy: 0.9609, test_Accuracy: 0.9502
Epoch: [ 0] [  303/  468] time: 64.5195, train_loss: 0.17442289, train_accuracy: 0.9688, test_Accuracy: 0.9503
Epoch: [ 0] [  304/  468] time: 64.7371, train_loss: 0.25443137, train_accuracy: 0.9375, test_Accuracy: 0.9502
E

Epoch: [ 0] [  370/  468] time: 79.4337, train_loss: 0.30737299, train_accuracy: 0.9219, test_Accuracy: 0.9558
Epoch: [ 0] [  371/  468] time: 79.6621, train_loss: 0.19801472, train_accuracy: 0.9453, test_Accuracy: 0.9545
Epoch: [ 0] [  372/  468] time: 79.8710, train_loss: 0.23308599, train_accuracy: 0.9531, test_Accuracy: 0.9548
Epoch: [ 0] [  373/  468] time: 80.0964, train_loss: 0.21890450, train_accuracy: 0.9453, test_Accuracy: 0.9547
Epoch: [ 0] [  374/  468] time: 80.2936, train_loss: 0.31902695, train_accuracy: 0.9453, test_Accuracy: 0.9541
Epoch: [ 0] [  375/  468] time: 80.4956, train_loss: 0.26726213, train_accuracy: 0.9453, test_Accuracy: 0.9531
Epoch: [ 0] [  376/  468] time: 80.7230, train_loss: 0.23596543, train_accuracy: 0.9531, test_Accuracy: 0.9526
Epoch: [ 0] [  377/  468] time: 80.9514, train_loss: 0.10353015, train_accuracy: 0.9844, test_Accuracy: 0.9535
Epoch: [ 0] [  378/  468] time: 81.1857, train_loss: 0.23048875, train_accuracy: 0.9297, test_Accuracy: 0.9538
E

Epoch: [ 0] [  444/  468] time: 95.3640, train_loss: 0.21963719, train_accuracy: 0.9766, test_Accuracy: 0.9610
Epoch: [ 0] [  445/  468] time: 95.5787, train_loss: 0.10748488, train_accuracy: 0.9766, test_Accuracy: 0.9611
Epoch: [ 0] [  446/  468] time: 95.8052, train_loss: 0.12777908, train_accuracy: 0.9609, test_Accuracy: 0.9610
Epoch: [ 0] [  447/  468] time: 96.0316, train_loss: 0.14790826, train_accuracy: 0.9766, test_Accuracy: 0.9611
Epoch: [ 0] [  448/  468] time: 96.2629, train_loss: 0.18258621, train_accuracy: 0.9688, test_Accuracy: 0.9611
Epoch: [ 0] [  449/  468] time: 96.4737, train_loss: 0.15016100, train_accuracy: 0.9766, test_Accuracy: 0.9606
Epoch: [ 0] [  450/  468] time: 96.6826, train_loss: 0.24035206, train_accuracy: 0.9531, test_Accuracy: 0.9602
Epoch: [ 0] [  451/  468] time: 96.8797, train_loss: 0.18951976, train_accuracy: 0.9609, test_Accuracy: 0.9601
Epoch: [ 0] [  452/  468] time: 97.0827, train_loss: 0.19344136, train_accuracy: 0.9688, test_Accuracy: 0.9599
E

### Test accuracy : 96.27%